In [ ]:
def call_amap_mcp_tool(params: dict) -> dict:
    """
    封装高德地图MCP Server的SSE调用逻辑
    :param params: MCP工具调用参数（如{"type": "geocode", "address": "北京市朝阳区望京SOHO"}）
    :return: 解析后的MCP响应结果
    """
    amap_config = MCP_SERVER_CONFIG["amap-maps"]
    if not amap_config["isActive"]:
        raise ValueError("高德地图MCP Server未激活")

    # 1. 按MCP协议封装SSE请求体（核心：符合Anthropic MCP规范）
    mcp_request = {
        "type": "tool_call",
        "tool_id": "amap-maps",  # 与MCP Server的标识一致
        "params": params,
        "context_ref": "amap_task_001"  # 上下文ID，关联多轮调用
    }

    # 2. 发起SSE连接（POST请求，流式接收响应）
    try:
        # 构建SSE请求（POST + JSON请求体 + 自定义头）
        response = requests.post(
            url=amap_config["baseUrl"],
            json=mcp_request,
            headers={**amap_config["headers"], "Content-Type": "application/json"},
            stream=True  # 流式响应（SSE核心）
        )
        response.raise_for_status()  # 检查HTTP状态码

        # 3. 解析SSE流响应
        client = sseclient.SSEClient(response)
        for event in client.events():
            if event.event == "message":  # SSE默认消息类型
                mcp_response = eval(event.data)  # 解析JSON字符串为字典（生产环境建议用json.loads）
                if mcp_response.get("type") == "tool_result":
                    if mcp_response["status"] == "success":
                        return mcp_response["result"]
                    else:
                        raise Exception(f"MCP工具调用失败：{mcp_response.get('error_msg', '未知错误')}")
        raise Exception("SSE流未返回有效工具结果")

    except Exception as e:
        return {"error": f"调用高德地图MCP工具失败：{str(e)}"}